![An interactive LADAL notebook](https://slcladal.github.io/images/uq1.jpg)

***

Please copy this Jupyter notebook so that you are able to edit it.

Simply go to: File > Save a copy in Drive.

Once you have done that, you are good to go.

***

This tutorial is the interactive Jupyter notebook accompanying the [*Language Technology and Data Analysis Laboratory* (LADAL) tutorial *Concordancing with R*](https://ladal.edu.au/kwics.html). 



***


**Preparation and session set up**

We set up our session by activating the packages we need for this tutorial.


In [ ]:
# activate packages
library(quanteda)
library(dplyr)
library(stringr)


Once you have initiated the session by executing the code shown above, you are good to go.

If you are using this notebook on your own computer and you have not already installed the R packages listed above, you need to install them. You can install them by replacing the `library` command with `install.packages` and putting the name of the package into quotation marks like this: `install.packages("quanteda")`. Then, you simply run this command and R will install the package you specified.


## Loading and processing textual data

For this tutorial, the default data represents the text of Lewis Caroll's  *Alice's Adventures in Wonderland* which we download from the [GitHub data repository of the *Language Technology and Data Analysis Laboratory* (LADAL)](https://slcladal.github.io/data). 

***

## Using your own data

While the tutorial uses data from the LADAL website, you can also use your own data. You can see below what you need to do to upload and use your own data.

The code chunk below allows you to upload two files from your own computer. To be able to load your own data, you need to click on the folder symbol to the left of the screen:

![Binder Folder Symbol](https://slcladal.github.io/images/binderfolder.JPG)


Then on the upload symbol.

![Binder Upload Symbol](https://slcladal.github.io/images/binderupload.JPG)

Next, upload the files you want to analyze and then the respective files names in the file argument of the scan function. When you then execute the code (like to code chunk below, you will upload your own data.


In [ ]:
mytext1 <- scan(file = "linguistics01.txt",
            what = "char", 
            sep = "", 
            quote = "", 
            quiet = T, 
            skipNul = T) %>%
            paste0(collapse = " ")
mytext2 <- scan(file = "linguistics02.txt",
            what = "char", 
            sep = "", 
            quote = "", 
            quiet = T, 
            skipNul = T) %>%
            paste0(collapse = " ")
# inspect
mytext1; mytext2


**Keep in mind though that you need to adapt the names of the texts in the code chunks below so that the code below work on your own texts!**

***

If you do not use your own data, you can continue with the default data, Lewis Caroll's  *Alice's Adventures in Wonderland*, which we load by running the code below (but you have to have access to the internet to do so).


In [ ]:
text <- base::readRDS(url("https://slcladal.github.io/data/alice.rda", "rb"))
# inspect
head(text)


The inspection of the data shows that the data consists of a vector of individual strings which contain the example text. This means that the data requires formatting so that we can use it. Therefore, we collapse it into a single object (or text),  remove superfluous white spaces, and then tokenize the data (tokenizing means that we split it into individual  tokens or words.



In [ ]:
text <- text %>%
  # collapse lines into a single  text
  paste0(collapse = " ") %>%
  # remove superfluous white spaces
  str_squish() %>%
  # tokenize
  tokens()
# inspect
head(text)


The result confirms that the entire text is now split into individual words. 

## Creating simple concordances

Now that we have loaded the data, we can easily extract concordances using the `kwic` function from the `quanteda` package. The `kwic` function takes the text (`x`) and the search pattern (`pattern`) as it main arguments but it also allows the specification of the context window, i.e. how many words/elements are show to the left and right of the key word (we will go over this later on).


In [ ]:
kwic_alice <- kwic(
  # define text
  text, 
  # define target word (this is called the "search pattern")
  pattern = "alice")
# inspect
kwic_alice %>%
  as.data.frame() %>%
  head()


We can easily extract the frequency of the search term (*alice*) using the `nrow` or the `length` functions which provide the number of rows of a tables (`nrow`) or the length of a vector (`length`).



In [ ]:
nrow(kwic_alice); length(kwic_alice$keyword)



The results show that there are 386 instances of the search term (*alice*). To get a better understanding of the use of a word, it is often useful to extract more context. This is easily done by increasing size of the context window. To do this, we specify the `window` argument of the `kwic` function. In the example below, we set the context window size to 10 words/elements rather than using the default (which is 5 word/elements).



In [ ]:
kwic_alice_longer <- kwic(text, pattern = "alice", 
  # define context window size
  window = 10)
# inspect
kwic_alice_longer %>%
  as.data.frame() %>%
  head()


## Extracting more than single words

While extracting single words is very common, you may want to extract more than just one word. To extract phrases, all you need to so is to specify that the pattern you are looking for is a phrase, as shown below.


In [ ]:
kwic_pooralice <- kwic(text, pattern = phrase("poor alice"))
# inspect
kwic_pooralice %>%
  as.data.frame() %>%
  head()


Of course you can extend this to longer sequences such as entire sentences. However, you may want to extract more or less concrete patterns rather than words or phrases. To search for patterns rather than words, you need to include regular expressions in your search pattern. 



## Searches using regular expressions

Regular expressions allow you to search for abstract patterns rather than concrete words or phrases which provides you with an extreme flexibility in what you can retrieve. A regular expression (in short also called *regex* or *regexp*) is a special sequence of characters that stand for are that describe a pattern. For more information about regular expression in R [see this tutorial](https://ladal.edu.au/regex.html).


To include regular expressions in your KWIC searches, you include them in your search pattern and set the argument `valuetype` to `"regex"`. The search pattern `"\\balic.*|\\bhatt.*"` retrieves elements that contain `alic` and `hatt` followed by any characters and where the `a` in `alic` and the `h` in `hatt` are at a word boundary, i.e. where they are the first letters of a word. Hence, our search would not retrieve words like *malice* or *shatter*. The `|` is an operator (like `+`, `-`, or `*`) that stands for *or*.


In [ ]:
# define search patterns
patterns <- c("\\balic.*|\\bhatt.*")
kwic_regex <- kwic(text, patterns, 
  # define valuetype
  valuetype = "regex")
# inspect
kwic_regex %>%
  as.data.frame() %>%
  head()


## Piping concordances

Quite often, we only want to retrieve patterns if they occur in a certain context. For instance, we might be interested in instances of *alice* but only if the preceding word is *poor*. Such conditional concordances could be extracted using regular expressions but they are easier to retrieve by piping. Piping is done using the `%>%` function from the `dplyr` package and the piping sequence can be translated as *and then*. We can then filter those concordances that contain *poor* using the `filter` function from the `dplyr` package. Note the the `$` stands for the end of a string so that *poor$* means that *poor* is the last element in the string that is preceding the keyword.


In [ ]:
kwic_pipe <- kwic(x = text, pattern = "alice") %>%
  dplyr::filter(stringr::str_detect(pre, "poor$|little$"))
# inspect
kwic_pipe %>%
  as.data.frame() %>%
  head()


Piping is a very useful helper function and it is very frequently used in R - not only in the context of text processing but in all data science related domains.

## Arranging concordances and adding frequency information

When inspecting concordances, it is useful to re-order the concordances so that they do not appear in the order that they appeared in the text or texts but by the context. To reorder concordances, we can use the `arrange` function from the `dplyr` package which takes the column according to which we want to re-arrange the data as it main argument. 

In the example below, we extract all instances of *alice* and then arrange the instances according to the content of the `post` column in alphabetical.


In [ ]:
kwic_ordered <- kwic(x = text, pattern = "alice") %>%
  dplyr::arrange(post)
# inspect
kwic_ordered %>%
  as.data.frame() %>%
  head() 


Arranging concordances according to alphabetical properties may, however, not be the most useful option. A more useful option may be to arrange concordances according to the frequency of co-occurring terms or collocates. In order to do this, we need to extract the co-occurring words and calculate their frequency. We can do this by combining the  `mutate`, `group_by`, `n()` functions from the `dplyr` package with the `str_remove_all` function from the `stringr` package. Then, we arrange the concordances by the frequency of the collocates in descending order (that is why we put a `-` in the arrange function). In order to do this, we need to 

1. create a new variable or column which represents the word that co-occurs with, or, as in the example below, immediately follows the search term. In the example below, we use the `mutate` function to create a new column called `post_word`. We then use the `str_remove_all` function to remove everything except for the word that immediately follows the search term (we simply remove everything and including a white space).

2. group the data by the word that immediately follows the search term.

3. create a new column called `post_word_freq` which represents the frequencies of all the words that immediately follow the search term.

4. arrange the concordances by the frequency of the collocates in descending order.


In [ ]:
kwic_ordered_coll <- kwic(
  # define text
  x = text, 
  # define search pattern
  pattern = "alice") %>%
  # extract word following the keyword
  dplyr::mutate(post_word = str_remove_all(post, " .*")) %>%
  # group following words
  dplyr::group_by(post_word) %>%
  # extract frequencies of the following words
  dplyr::mutate(post_word_freq = n()) %>%
  # arrange/order by the frequency of the following word
  dplyr::arrange(-post_word_freq)
# inspect
kwic_ordered_coll %>%
  as.data.frame() %>%
  head()


We add more columns according to which we could arrange the concordance following the same schema. For example, we could add another column that represented the frequency of words that immediately preceded the search term and then arrange according to this column.

## Ordering by subsequent elements

In this section, we will extract the three words following the keyword (*alice*) and organize the concordances by the frequencies of the following words. We begin by inspecting the first 6 lines of the concordance of *alice*.


In [ ]:
head(kwic_alice)



Next, we take the concordances and create a clean post column that is all in lower case and that does not contain any punctuation.



In [ ]:
kwic_alice %>%
  # convert to data frame
  as.data.frame() %>%
  # create new CleanPost
  dplyr::mutate(CleanPost = stringr::str_remove_all(post, "[:punct:]"),
                CleanPost = stringr::str_squish(CleanPost),
                CleanPost = tolower(CleanPost))-> kwic_alice_following
# inspect
head(kwic_alice_following)


In a next step, we extract the 1^st^, 2^nd^, and 3^rd^ words following the keyword.



In [ ]:
kwic_alice_following %>%
  # extract first element after keyword
  dplyr::mutate(FirstWord = stringr::str_remove_all(CleanPost, " .*")) %>%
  # extract second element after keyword
  dplyr::mutate(SecWord = stringr::str_remove(CleanPost, ".*? "),
                SecWord = stringr::str_remove_all(SecWord, " .*")) %>%
  # extract third element after keyword
  dplyr::mutate(ThirdWord = stringr::str_remove(CleanPost, ".*? "),
                ThirdWord = stringr::str_remove(ThirdWord, ".*? "),
                ThirdWord = stringr::str_remove_all(ThirdWord, " .*")) -> kwic_alice_following
# inspect
head(kwic_alice_following)


Next, we calculate the frequencies of the subsequent words and order in descending order from the  1^st^ to the 3^rd^ word following the keyword.



In [ ]:
kwic_alice_following %>%
  # calculate frequency of following words
  # 1st word
  dplyr::group_by(FirstWord) %>%
  dplyr::mutate(FreqW1 = n()) %>%
  # 2nd word
  dplyr::group_by(SecWord) %>%
  dplyr::mutate(FreqW2 = n()) %>%
  # 3rd word
  dplyr::group_by(ThirdWord) %>%
  dplyr::mutate(FreqW3 = n()) %>%
  # ungroup
  dplyr::ungroup() %>%
  # arrange by following words
  dplyr::arrange(-FreqW1, -FreqW2, -FreqW3) -> kwic_alice_following
# inspect results
head(kwic_alice_following, 10)


The results now show the concordance arranged by the frequency of the words following the keyword.

## Concordances from transcriptions

As many analyses in the language sciences use transcripts as their primary data and because transcripts have features that require additional processing, we will now perform concordancing based on on transcripts. As a first step, we load five example transcripts that represent the first five files from the Irish component of the [International Corpus of English](https://www.ice-corpora.uzh.ch/en.html).


In [ ]:
# define corpus files
files <- paste("https://slcladal.github.io/data/ICEIrelandSample/S1A-00", 1:5, ".txt", sep = "")
# load corpus files
transcripts <- sapply(files, function(x){
  x <- readLines(x)
  })
# inspect
transcripts[[1]][1:10] %>%
  as.data.frame()


The first ten lines shown above let us know that, after the header (`<S1A-001 Riding>`) and the symbol which indicates the start of the transcript (`<I>`), each utterance is preceded by a sequence which indicates the section, file, and speaker (e.g. `<S1A-001$A>`). The first utterance is thus uttered by speaker `A` in file `001` of section `S1A`. In addition, there are several sequences that provide meta-linguistic information which indicate the beginning of a speech unit (`<#>`), pauses (`<,>`), and laughter (`<&> laughter </&>`).

To perform the concordancing, we need to change the format of the transcripts because the `kwic` function only works on character, corpus, tokens object- in their present form, the transcripts represent a list which contains vectors of strings. To change the format, we collapse the individual utterances into a single character vector for each transcript.


In [ ]:
transcripts_collapsed <- sapply(files, function(x){
  # read-in text
  x <- readLines(x)
  # paste all lines together
  x <- paste0(x, collapse = " ")
  # remove superfluous white spaces
  x <- str_squish(x)
})
# inspect
transcripts_collapsed %>%
    substr(start=1, stop=500) %>%
  as.data.frame()


We can now extract the concordances.



In [ ]:
kwic_trans <- quanteda::kwic(tokens(transcripts_collapsed), pattern = phrase("you know"))
# inspect
kwic_trans %>%
  as.data.frame() %>%
  head() 


The results show that each non-alphanumeric character is counted as a single word which reduces the context of the keyword substantially. Also, the *docname* column contains the full path to the data which make it hard to parse the content of the table. To address the first issue, we specify the tokenizer that we will use to not disrupt the annotation too much. In addition, we clean the *docname* column and extract only the file name. Lastly, we will expand the context window to 10 so that we have a better understanding of the context in which the phrase was used.



In [ ]:
kwic_trans <- quanteda::kwic(tokens(transcripts_collapsed, what = "fasterword"), 
  pattern = phrase("you know"), window = 10) %>%
  # clean docnames
  dplyr::mutate(docname = str_replace_all(docname, ".*/([A-Z][0-9][A-Z]-[0-9]{1,3}).txt", "\\1"))
# inspect
kwic_trans %>%
  as.data.frame() %>%
  head()


[Back to LADAL](https://ladal.edu.au/kwics.html)

